In [ ]:
import heapq
import matplotlib.pyplot as plt
import numpy as np

# Manhattan distance
def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* search
def astar(maze, start, goal, heuristic):
    rows, cols = maze.shape
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1], g_score[goal]

        for dx, dy in [(0,1),(1,0),(0,-1),(-1,0)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if maze[neighbor] == 1:
                    continue
                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))
    return None, float("inf")

# Maze
maze = [
    ['A', 0,1,0,0,0,1,0,0,1,0,0],
    [0,0,1,0,1,0,1,0,1,0,0,0],
    [1,0,0,0,1,0,0,0,1,1,1,0],
    [0,1,1,0,0,1,1,0,0,0,1,0],
    [0,0,0,0,1,0,1,1,1,0,0,0],
    [1,1,1,0,1,0,0,0,1,1,1,0],
    [0,0,1,0,0,0,1,0,0,0,1,0],
    [0,1,0,1,1,0,1,1,1,0,1,0],
    [0,0,0,0,0,0,0,0,1,0,0,0],
    [1,1,1,1,1,1,0,1,0,1,1,0],
    [0,0,0,0,0,0,0,1,0,0,0,0],
    [0,1,1,1,1,1,1,1,1,1,1,'B']
]

# Convert
maze_np = np.zeros((len(maze5), len(maze5[0])), dtype=int)
start, goal = None, None
for i in range(len(maze5)):
    for j in range(len(maze5[0])):
        if maze5[i][j] == 'A':
            start = (i, j); maze_np[i,j] = 0
        elif maze5[i][j] == 'B':
            goal = (i, j); maze_np[i,j] = 0
        else:
            maze_np[i,j] = maze5[i][j]

# Run
path, cost = astar(maze_np, start, goal, manhattan)
print("Path:", path)
print("Cost:", cost)

# Visualization
plt.figure(figsize=(7,7))
plt.imshow(maze_np, cmap="binary")
if path:
    px, py = zip(*path)
    plt.plot(py, px, marker="o", color="red", linewidth=2)
plt.scatter(start[1], start[0], color="green", s=100, label="Start")
plt.scatter(goal[1], goal[0], color="blue", s=100, label="Goal")
plt.legend(); plt.title("A* with Manhattan Distance"); plt.show()


In [ ]:
# Case 1: Manhattan × 1.5 (may not be admissible)
def scaled_manhattan(a, b):
    return 1.5 * manhattan(a, b)

# Case 2: Inconsistent heuristic
def inconsistent_heuristic(a, b):
    if a == (2, 2):   # one node made inconsistent
        return manhattan(a, b) + 3
    return manhattan(a, b)

# Run case 1
path1, cost1 = astar(maze_np, start, goal, scaled_manhattan)
print("\nCase 1 (Scaled Manhattan x1.5):")
print("Path:", path1)
print("Cost:", cost1)

# Run case 2
path2, cost2 = astar(maze_np, start, goal, inconsistent_heuristic)
print("\nCase 2 (Inconsistent heuristic):")
print("Path:", path2)
print("Cost:", cost2)


In [ ]:
from collections import deque

def bfs_shortest(maze, start, goal):
    rows, cols = maze.shape
    queue = deque([(start, [start])])
    visited = {start}
    while queue:
        current, path = queue.popleft()
        if current == goal:
            return path, len(path)-1
        for dx, dy in [(0,1),(1,0),(0,-1),(-1,0)]:
            neighbor = (current[0]+dx, current[1]+dy)
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if maze[neighbor]==0 and neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path+[neighbor]))
    return None, float("inf")

# True shortest
true_path, true_cost = bfs_shortest(maze_np, start, goal)
print("\nTrue shortest cost (BFS):", true_cost)

# Compare all
print("\n--- Comparison ---")
print("Manhattan:", cost, "Optimal?", cost==true_cost)
print("Scaled Manhattan:", cost1, "Optimal?", cost1==true_cost)
print("Inconsistent:", cost2, "Optimal?", cost2==true_cost)
